In [29]:
from braininvaders2015a.dataset import BrainInvaders2015a
import pandas as pd
import warnings
import mne
import numpy as np
from mne import create_info
from mne.io import RawArray
import torch
import torch.nn as nn
import torch.nn.functional as F
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
import torch.optim as optim
import csv
warnings.filterwarnings("ignore")

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [26]:
dataset = BrainInvaders2015a()
# Set up hyper parameter
EVENT_ID = {'NonTarget': 1, 'Target': 2}
TMIN = 0.0
TMAX = 0.5
TEST_RATIO = 0.2
VAL_RATIO = 0.1
BATCH_SIZE = 10

N_CLASS = 3

LEARNING_RATE = 0.001
NUM_EPOCH = 20
SUBJECT_LIST = np.delete(np.arange(44), [0, 1, 27])

[ 2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25
 26 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43]


In [3]:
def df_to_raw(df):
    sfreq = 512
    ch_names = list(df.columns)
    ch_types = ['eeg'] * (len(df.columns) - 1) + ['stim']
    ten_twenty_montage = mne.channels.make_standard_montage('standard_1020')

    df = df.T
      #mne looks at the tranpose() format
    df[:-1] *= 1e-6
      #convert from uVolts to Volts (mne assumes Volts data)

    info = create_info(ch_names=ch_names, ch_types=ch_types, sfreq=sfreq)
    raw = mne.io.RawArray(df, info)
    raw.set_montage(ten_twenty_montage)

    return raw

In [4]:
def getRawSubjectById(id):
    # Get subject by id
    subject_id = id
    data_subject1 = dataset._get_single_subject_data(subject_id)
    # Raw to panda
    data_ses1_run1 = data_subject1['session_1']['run_1']
    data_ses1_run1_pd = data_ses1_run1.to_data_frame()
    data_ses1_run1_pd.head()
    # Drop time column
    data_ses1_run1_pd = data_ses1_run1_pd.drop(['time'],axis = 1)
    raw = df_to_raw(data_ses1_run1_pd)
    return raw

In [5]:
def preprocessing(raw):
    # 50hz notch filter
    raw.notch_filter(np.arange(50, 251, 50))
    # Read eeg chanels
    eeg_channels = mne.pick_types(raw.info, eeg=True)
    # Cut low frequency
    for cutoff in (0.1, 0.2, 1):
        raw_highpass = raw.copy().filter(l_freq=cutoff, h_freq=None)

    # Band pass 
    raw.filter(1,24,method = 'iir')
    # Split target
    raw1 = raw.to_data_frame() 
    X1 = raw1.drop(['time'],axis = 1)
    X = X1.drop(['STI 014'],axis = 1)
    y = raw1['STI 014']
    # PCA
    pca = PCA(n_components=32)
    X_r = pca.fit(X.values).transform(X.values)
    y1 = y.values.reshape(-1,1)
    data = np.hstack((X_r,y1))
    df = pd.DataFrame(data, columns = list(X1.columns))
    # Convert dataframe into raw
    result_raw = df_to_raw(df)
    return result_raw, eeg_channels

In [6]:
from mne import Epochs, find_events

def getEpochs(raw, event_id, tmin, tmax, picks):

    #epoching
    events = find_events(raw)
    print(events)

    reject_criteria = dict(eeg=100e-6)  #most voltage in this range is not brain components

    epochs = Epochs(raw, events=events, event_id=event_id, 
                    tmin=tmin, tmax=tmax, baseline=None, preload=True,verbose=False, picks=picks)

    return epochs

In [7]:
def eeg_braininvader_3d_mapping(X):
    n_sample = X.shape[0]
    chanel = X.shape[1]
    n_sampling = X.shape[2]
    X_3d = np.zeros((n_sample, n_sampling, 10, 11))
    for sample_idx in range(n_sample):
        for sampling_idx in range(n_sampling):
            X_3d[sample_idx, sampling_idx, 0, 4] = X[sample_idx, 0, sampling_idx]
            X_3d[sample_idx, sampling_idx, 0, 6] = X[sample_idx, 1, sampling_idx]
            X_3d[sample_idx, sampling_idx, 1, 5] = X[sample_idx, 2, sampling_idx]
            X_3d[sample_idx, sampling_idx, 2, 1] = X[sample_idx, 3, sampling_idx]
            X_3d[sample_idx, sampling_idx, 2, 3] = X[sample_idx, 4, sampling_idx]
            X_3d[sample_idx, sampling_idx, 2, 7] = X[sample_idx, 5, sampling_idx]
            X_3d[sample_idx, sampling_idx, 2, 9] = X[sample_idx, 6, sampling_idx]
            X_3d[sample_idx, sampling_idx, 3, 2] = X[sample_idx, 7, sampling_idx]
            X_3d[sample_idx, sampling_idx, 3, 4] = X[sample_idx, 8, sampling_idx]
            X_3d[sample_idx, sampling_idx, 3, 6] = X[sample_idx, 9, sampling_idx]
            X_3d[sample_idx, sampling_idx, 3, 8] = X[sample_idx, 10, sampling_idx]
            X_3d[sample_idx, sampling_idx, 4, 1] = X[sample_idx, 11, sampling_idx]
            X_3d[sample_idx, sampling_idx, 4, 3] = X[sample_idx, 12, sampling_idx]
            X_3d[sample_idx, sampling_idx, 4, 5] = X[sample_idx, 13, sampling_idx]
            X_3d[sample_idx, sampling_idx, 4, 7] = X[sample_idx, 14, sampling_idx]
            X_3d[sample_idx, sampling_idx, 4, 9] = X[sample_idx, 15, sampling_idx]
            X_3d[sample_idx, sampling_idx, 5, 2] = X[sample_idx, 16, sampling_idx]
            X_3d[sample_idx, sampling_idx, 5, 4] = X[sample_idx, 17, sampling_idx]
            X_3d[sample_idx, sampling_idx, 5, 6] = X[sample_idx, 18, sampling_idx]
            X_3d[sample_idx, sampling_idx, 5, 8] = X[sample_idx, 19, sampling_idx]
            X_3d[sample_idx, sampling_idx, 6, 1] = X[sample_idx, 20, sampling_idx]
            X_3d[sample_idx, sampling_idx, 6, 3] = X[sample_idx, 21, sampling_idx]
            X_3d[sample_idx, sampling_idx, 6, 5] = X[sample_idx, 22, sampling_idx]
            X_3d[sample_idx, sampling_idx, 6, 7] = X[sample_idx, 23, sampling_idx]
            X_3d[sample_idx, sampling_idx, 6, 9] = X[sample_idx, 24, sampling_idx]
            X_3d[sample_idx, sampling_idx, 8, 1] = X[sample_idx, 25, sampling_idx]
            X_3d[sample_idx, sampling_idx, 8, 3] = X[sample_idx, 26, sampling_idx]
            X_3d[sample_idx, sampling_idx, 8, 5] = X[sample_idx, 27, sampling_idx]
            X_3d[sample_idx, sampling_idx, 8, 7] = X[sample_idx, 28, sampling_idx]
            X_3d[sample_idx, sampling_idx, 8, 9] = X[sample_idx, 29, sampling_idx]
            X_3d[sample_idx, sampling_idx, 9, 2] = X[sample_idx, 30, sampling_idx]
            X_3d[sample_idx, sampling_idx, 9, 8] = X[sample_idx, 31, sampling_idx]
    return X_3d

In [8]:
def trainTestValSplit(X, y, test_ratio=0.2, val_ratio=0.1):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_ratio, random_state=42)
    valid_size = int(X.shape[0] * val_ratio)
    X_val = X_train[:valid_size, :]
    y_val = y_train[:valid_size]
    X_train = X_train[valid_size:, :]
    y_train = y_train[valid_size:]
    return X_train, X_val, X_test, y_train, y_val, y_test


In [9]:
def getTrainLoader(X_train, y_train, batch_size):
    train_dataset = [[X_train[i,:], y_train[i]] for i in range(X_train.shape[0])]
    train_loader = torch.utils.data.DataLoader(train_dataset, shuffle=True, batch_size=batch_size)
    return train_loader

In [10]:
# Convolutional neural network (two convolutional layers)
class Conv3DNet(nn.Module):
    def __init__(self, num_classes=10):
        super(Conv3DNet, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(in_channels=257, out_channels=150, kernel_size=(3,3), stride=1, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(150),
            nn.Dropout2d(p=0.2)
        )
        self.layer2 = nn.Sequential(
            nn.Conv2d(in_channels=150, out_channels=75, kernel_size=(3,3), stride=1, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(75),
            nn.Dropout2d(p=0.2)
        )
        self.layer3 = nn.Sequential(
            nn.Conv2d(in_channels=75, out_channels=10, kernel_size=(3,3), stride=1, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(10),
            nn.Dropout2d(p=0.2)
        )
        self.avgPool = nn.Sequential(
            nn.AvgPool2d(kernel_size=(2,2), stride=1),
            nn.BatchNorm2d(10)
        )

        self.fullyCon1 = nn.Sequential(
            nn.Linear(900,450),
            nn.Sigmoid(),
            nn.Dropout(p=0.2)
        )
        self.fullyCon2 = nn.Sequential(
            nn.Linear(450,100),
            nn.Sigmoid()
        )
        self.fullyCon3 = nn.Sequential(
            nn.Linear(100,num_classes)
        )


    def forward(self, x):
        # CNN
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.avgPool(x)

        # Fully Connected model
        x = x.reshape(x.size(0), -1)
        x = self.fullyCon1(x)
        x = self.fullyCon2(x)
        x = self.fullyCon3(x)
    
        return x

In [11]:
def train(train_loader, X_train, X_val, y_train, y_val, num_classes, learning_rate, num_epochs):
    model = Conv3DNet(num_classes=num_classes).to(device).double()
    print(model)
    
    Xt_val = torch.DoubleTensor(X_val)
    Xt_train = torch.DoubleTensor(X_train)

    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

    if torch.cuda.is_available():
        Xt_val = Xt_val.cuda()
        Xt_train = Xt_train.cuda() 

    # Train the model
    total_step = len(train_loader)

    for epoch in range(num_epochs):
        epoch_loss = 0
        model.train()
        for i, (data, target) in enumerate(train_loader):
            
            if torch.cuda.is_available():
                data = data.to(device)
                target = target.to(device)
            
            # Forward pass
            outputs = model(data)
            
            loss = criterion(outputs, target)
            
            # Backward and optimize
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            epoch_loss += loss.item()

        model.eval()
        
        train_predict = model(Xt_train)
        train_classes = torch.argmax(train_predict, axis=1)
        train_correct = [1 if train_classes[i] == y_train[i] else 0 for i in range(train_classes.shape[0])]
        train_acc = sum(train_correct)/len(train_correct)
        
        val_predict = model(Xt_val)
        val_classes = torch.argmax(val_predict, axis=1)
        val_correct = [1 if val_classes[i] == y_train[i] else 0 for i in range(val_classes.shape[0])]
        val_acc = sum(val_correct)/len(val_correct)
        
        print('\rEpoch [{}/{}], Training Loss: {:.4f}, Training Acc: {:.4f}, Val Acc: {:.4f}' 
                .format(epoch+1, num_epochs, epoch_loss, train_acc, val_acc))
        
    return model

In [12]:
def test(model, X_test, y_test):
    model.eval()
    
    Xt_test = torch.DoubleTensor(X_test)

    if torch.cuda.is_available():
        Xt_test = Xt_test.cuda()

    test_predict = model(Xt_test)
    test_classes = torch.argmax(test_predict, axis=1)
    test_correct = [1 if test_classes[i] == y_train[i] else 0 for i in range(test_classes.shape[0])]
    test_acc = sum(test_correct)/len(test_correct)

    print('\rTest Acc: {:.4f}' .format(test_acc))
    return test_acc

In [31]:
index = ['subject', 'accuracy']
test_accs = []
for i in SUBJECT_LIST:
    raw = getRawSubjectById(i)
    raw_preprocessed, eeg_channels = preprocessing(raw)
    picks = eeg_channels
    epochs = getEpochs(raw_preprocessed, EVENT_ID, TMIN, TMAX, picks)
    X = epochs.get_data()
    y = epochs.events[:, -1]

    X_3d = eeg_braininvader_3d_mapping(X)
    X_train, X_val, X_test, y_train, y_val, y_test = trainTestValSplit(X_3d, y, TEST_RATIO, VAL_RATIO)

    train_loader = getTrainLoader(X_train, y_train, BATCH_SIZE)

    model = train(train_loader, X_train, X_val, y_train, y_val, N_CLASS, LEARNING_RATE, 30)

    test_accuracy = test(model, X_test, y_test)
    test_accs.append([i, test_accuracy])

with open('testAccuracy.csv', 'w') as f:
    write = csv.writer(f)
      
    write.writerow(index)
    write.writerows(test_accs)

Creating RawArray with float64 data, n_channels=33, n_times=177248
    Range : 0 ... 177247 =      0.000 ...   346.186 secs
Ready.
Setting up band-stop filter

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandstop filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower transition bandwidth: 0.50 Hz
- Upper transition bandwidth: 0.50 Hz
- Filter length: 3381 samples (6.604 sec)

Filtering raw data in 1 contiguous segment
Setting up high-pass filter at 0.1 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal highpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.10
- Lower transition bandwidth: 0.10 Hz (-6 dB cutoff frequency: 0.05 Hz)
- Filter length: 16897 samples (33.002 sec)

Filtering raw data in 1 contiguous seg

100%|██████████| Downloading : 87.7M/87.7M [00:36<00:00,    2.54MB/s]


unzip /home/maxpasit/mne_data/MNE-braininvaders2015a-data/record/3266930/files/subject_04_mat.zip
Creating RawArray with float64 data, n_channels=33, n_times=134272
    Range : 0 ... 134271 =      0.000 ...   262.248 secs
Ready.
Setting up band-stop filter

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandstop filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower transition bandwidth: 0.50 Hz
- Upper transition bandwidth: 0.50 Hz
- Filter length: 3381 samples (6.604 sec)

Filtering raw data in 1 contiguous segment
Setting up high-pass filter at 0.1 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal highpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.10
- Lower transition bandwidth: 0.10 Hz (-6 dB cutoff frequ

100%|██████████| Downloading : 82.0M/82.0M [00:38<00:00,    2.21MB/s]


unzip /home/maxpasit/mne_data/MNE-braininvaders2015a-data/record/3266930/files/subject_05_mat.zip
Creating RawArray with float64 data, n_channels=33, n_times=150528
    Range : 0 ... 150527 =      0.000 ...   293.998 secs
Ready.
Setting up band-stop filter

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandstop filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower transition bandwidth: 0.50 Hz
- Upper transition bandwidth: 0.50 Hz
- Filter length: 3381 samples (6.604 sec)

Filtering raw data in 1 contiguous segment
Setting up high-pass filter at 0.1 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal highpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.10
- Lower transition bandwidth: 0.10 Hz (-6 dB cutoff frequ

...

unzip /home/maxpasit/mne_data/MNE-braininvaders2015a-data/record/3266930/files/subject_41_mat.zip
Creating RawArray with float64 data, n_channels=33, n_times=157376
    Range : 0 ... 157375 =      0.000 ...   307.373 secs
Ready.
Setting up band-stop filter

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandstop filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower transition bandwidth: 0.50 Hz
- Upper transition bandwidth: 0.50 Hz
- Filter length: 3381 samples (6.604 sec)

Filtering raw data in 1 contiguous segment
Setting up high-pass filter at 0.1 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal highpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.10
- Lower transition bandwidth: 0.10 Hz (-6 dB cutoff frequ

100%|██████████| Downloading : 152M/152M [01:22<00:00,    1.92MB/s]


unzip /home/maxpasit/mne_data/MNE-braininvaders2015a-data/record/3266930/files/subject_42_mat.zip
Creating RawArray with float64 data, n_channels=33, n_times=217504
    Range : 0 ... 217503 =      0.000 ...   424.811 secs
Ready.
Setting up band-stop filter

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandstop filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower transition bandwidth: 0.50 Hz
- Upper transition bandwidth: 0.50 Hz
- Filter length: 3381 samples (6.604 sec)

Filtering raw data in 1 contiguous segment
Setting up high-pass filter at 0.1 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal highpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.10
- Lower transition bandwidth: 0.10 Hz (-6 dB cutoff frequ

100%|██████████| Downloading : 88.9M/88.9M [00:52<00:00,    1.78MB/s]


unzip /home/maxpasit/mne_data/MNE-braininvaders2015a-data/record/3266930/files/subject_43_mat.zip
Creating RawArray with float64 data, n_channels=33, n_times=129472
    Range : 0 ... 129471 =      0.000 ...   252.873 secs
Ready.
Setting up band-stop filter

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandstop filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower transition bandwidth: 0.50 Hz
- Upper transition bandwidth: 0.50 Hz
- Filter length: 3381 samples (6.604 sec)

Filtering raw data in 1 contiguous segment
Setting up high-pass filter at 0.1 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal highpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.10
- Lower transition bandwidth: 0.10 Hz (-6 dB cutoff frequ